In [2]:

#comment out for local testing out of Main Sequence Platform
import dotenv
dotenv.load_dotenv('../../.env')

from mainsequence.virtualfundbuilder.portfolio_interface import PortfolioInterface
from mainsequence.virtualfundbuilder.agent_interface import TDAGAgent

2025-03-24T15:09:02.141802Z [error    ] Could not retrive pod project Project ID not found, server response {'detail': 'Invalid token.'} application_name=ms-sdk
Traceback (most recent call last):
  File "/home/jose/code/MainSequenceClientSide/mainsequence-sdk/mainsequence/logconf.py", line 191, in build_application_logger
    raise ValueError(f"Project ID not found, server response {json_response}")
ValueError: Project ID not found, server response {'detail': 'Invalid token.'} (at logconf.py:198 in build_application_logger())


Got Status Code 403 with response {"detail":"Invalid token."}


ValueError: Project ID not found, server response {'detail': 'Invalid token.'}

# Connect to the TDAG-LLM Agent

In [2]:
tdag_agent = TDAGAgent()

2025-03-17T20:04:11.881329Z [debug    ] Send default documentation to Backend application_name=ms-sdk data_source_id=1 job_run_id=None project_id=3 (at __main__.py:173 in send_default_configuration())
2025-03-17T20:04:11.906255Z [debug    ] Requesting POST from http://127.0.0.1:8000/orm/api/tdag-gpt/register_default_configuration/ application_name=ms-sdk data_source_id=1 job_run_id=None project_id=3 (at utils.py:55 in make_request())
2025-03-17T20:04:12.479395Z [info     ] Setup TDAG Agent successfull   application_name=ms-sdk data_source_id=1 job_run_id=None project_id=3 (at agent_interface.py:28 in __init__())


# Create a custom Strategy

# Get the configuration for the full portfolio based on our strategy

In [3]:
PortfolioInterface.list_configurations()

['FixedWeights_configuration',
 'MarketCapCryptoSpot_configuration',
 'MarketCapEquities_configuration',
 'MarketCap_configuration']

In [4]:
from mainsequence.virtualfundbuilder.contrib.signals import MarketCap

portfolio = tdag_agent.generate_portfolio(MarketCap, signal_description="Create me a market cap portfolio using AAPL and GOOG")

portfolio

2025-03-17T20:04:12.761170Z [debug    ] Register signal: MarketCap     application_name=ms-sdk data_source_id=1 job_run_id=None project_id=3 (at utils.py:675 in _send_strategy_to_registry())
2025-03-17T20:04:12.781214Z [debug    ] Requesting POST from http://127.0.0.1:8000/orm/api/tdag-gpt/register_strategy/ application_name=ms-sdk data_source_id=1 job_run_id=None project_id=3 (at utils.py:55 in make_request())
2025-03-17T20:04:13.299679Z [info     ] Get configuration for MarketCap ... application_name=ms-sdk data_source_id=1 job_run_id=None project_id=3 (at agent_interface.py:43 in generate_portfolio())
2025-03-17T20:04:13.312259Z [debug    ] Requesting POST from http://127.0.0.1:8000/orm/api/tdag-gpt/create_configuration_for_strategy/ application_name=ms-sdk data_source_id=1 job_run_id=None project_id=3 (at utils.py:55 in make_request())
2025-03-17T20:04:49.364836Z [info     ] Replaced 'None' in configuration with None at root.portfolio_build_configuration.backtesting_weights_configu

Configuration Name: -
portfolio_build_configuration:
  assets_configuration:
    asset_universe:
      asset_filters:
      - asset_type: cash_equity
        execution_venue__symbol: alpaca
        symbol: AAPL
      - asset_type: cash_equity
        execution_venue__symbol: alpaca
        symbol: GOOG
    price_type: vwap
    prices_configuration:
      bar_frequency_id: 1d
      intraday_bar_interpolation_rule: ffill
      upsample_frequency_id: 1d
  backtesting_weights_configuration:
    rebalance_strategy_configuration:
      calendar: 24/7
    rebalance_strategy_name: ImmediateSignal
    signal_weights_configuration:
      num_top_assets: None
      signal_assets_configuration:
        asset_universe:
          asset_filters:
          - asset_type: cash_equity
            execution_venue__symbol: alpaca
            symbol: AAPL
          - asset_type: cash_equity
            execution_venue__symbol: alpaca
            symbol: GOOG
        price_type: vwap
        prices_configura

# Run the strategy

In [ ]:
res = portfolio.run()
res.head()

2025-03-17T20:04:50.076119Z [debug    ] Requesting GET from http://127.0.0.1:8000/orm/api/data_sources/historical-bars-source/ application_name=ms-sdk data_source_id=1 job_run_id=None project_id=3 (at utils.py:55 in make_request())


# Plot the portfolio results

In [ ]:
import plotly.graph_objects as go
import pandas as pd

# Create the plot
fig = go.Figure()

fig.add_trace(
    go.Scatter(
        x=res.index, 
        y=res["close"], 
        mode="lines+markers",
        name="Portfolio Value",
        line=dict(color='blue', width=1),
        marker=dict(size=2)
    )
)

# Customize layout
fig.update_layout(
    title="Portfolio Value Over Time",
    xaxis_title="Date",
    yaxis_title="Portfolio Value",
    template="plotly_white",
    hovermode="x unified"
)

# Show the plot
fig.show()